In [1]:
# 1. Csomagok importálása
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
import mlflow
import os

/home/luca/Documents/hazifeladatok/MSc/MLSecOps/MLSecOps-project/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-06 13:28:14.798051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746530894.829618  117136 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746530894.838359  117136 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746530894.860217  117136 computation_placer.cc:177] computation placer already registered. Please check linkage and a

In [ ]:
# 2. Adat betöltése (IMDB, 1000 minta a gyors teszt kedvéért)
dataset = load_dataset("imdb", split="train[:100]")
texts = dataset["text"]
labels = dataset["label"]  # 0 = negatív, 1 = pozitív

# 3. Sentiment pipeline betöltése
model = pipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english", task="sentiment-analysis"
)

# 4. Predikciók futtatása
# Add truncation and padding to handle varying sequence lengths
results = model(texts, truncation=True, padding=True)

# 5. Eredmények feldolgozása
pred_labels = [1 if r["label"] == "POSITIVE" else 0 for r in results]

# 6. Pontosság kiértékelése
from sklearn.metrics import accuracy_score

acc = accuracy_score(labels, pred_labels)
print(f"Pontosság: {acc:.4f}")

# 7. MLflow loggolás
mlflow.set_experiment("sentiment_analysis")

with mlflow.start_run():
    mlflow.log_param("model", "distilbert-base-uncased-finetuned-sst-2-english")
    mlflow.log_metric("accuracy", acc)

    # Modell mentése későbbi betöltéshez
    save_path = "model/"
    model.save_pretrained(save_path)
    mlflow.log_artifacts(save_path, artifact_path="model")

# 8. Predikciók mentése CSV-be (baseline számára)
df_preds = pd.DataFrame(
    {"text": texts, "true_label": labels, "pred_label": pred_labels}
)
df_preds.to_csv("predictions/baseline_predictions.csv", index=False)

/home/luca/Documents/hazifeladatok/MSc/MLSecOps/MLSecOps-project/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-05 11:49:33.621687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746438573.789626   56231 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746438573.823978   56231 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746438574.261279   56231 computation_placer.cc:177] computation placer already registered. Please check linkage and a

Pontosság: 0.9300
